# Проект e-commerce: вариант 1

1. Сколько у нас пользователей, которые совершили покупку только один раз? 

2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? 

3. По каждому товару определить, в какой день недели товар чаще всего покупается. 

4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. 

5. Напиши функцию на python, позволяющую строить когортный анализ. В период с января по декабрь выяви когорту с самым высоким retention на 3-й месяц. 

6. Построй RFM-кластеры для пользователей. Выведи для каждого кластера средние значения метрик R, F, M (и опиши подробно, как были построены метрики R, F, M). 


Файлы:

olist_customers_datase.csv — таблица с уникальными идентификаторами пользователей:

- customer_id — позаказный идентификатор пользователя (аналог номера паспорта)
- customer_unique_id —  уникальный идентификатор пользователя
- customer_zip_code_prefix —  почтовый индекс пользователя
- customer_city —  город доставки пользователя
- customer_state —  штат доставки пользователя

olist_orders_dataset.csv —  таблица заказов:

- order_id —  уникальный идентификатор заказа (номер чека)
- customer_id —  позаказный идентификатор пользователя (аналог номера паспорта)
- order_status —  статус заказа
- order_purchase_timestamp —  время создания заказа
- order_approved_at —  время подтверждения оплаты заказа
- order_delivered_carrier_date —  время передачи заказа в логистическую службу
- order_delivered_customer_date —  время доставки заказа
- order_estimated_delivery_date —  обещанная дата доставки

olist_order_items_dataset.csv —  товарные позиции, входящие в заказы:

- order_id —  уникальный идентификатор заказа (номер чека)
- order_item_id —  идентификатор товара внутри одного заказа
- product_id —  ид товара (аналог штрихкода)
- seller_id — ид производителя товара
- shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике
- price —  цена за единицу товара
- freight_value —  вес товара


In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns 
import datetime as dt 
from operator import attrgetter

### Данные

In [2]:
customers_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-prohorova-18/file/olist_customers_dataset.csv', encoding='Windows-1251', sep=',')

In [3]:
orders_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-prohorova-18/file/olist_orders_dataset.csv', encoding='Windows-1251', sep=',', parse_dates=[3,4,5,6,7])

In [4]:
order_items_data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-prohorova-18/file/olist_order_items_dataset.csv', encoding='Windows-1251', sep=',', parse_dates=[4])

In [13]:
customers_data.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


In [80]:
customers_data.shape

(99441, 5)

In [16]:
customers_data.nunique()

customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64

In [17]:
customers_data.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [14]:
orders_data.head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13


In [19]:
orders_data.nunique()

order_id                         99441
customer_id                      99441
order_status                         8
order_purchase_timestamp         98875
order_approved_at                90733
order_delivered_carrier_date     81018
order_delivered_customer_date    95664
order_estimated_delivery_date      459
dtype: int64

In [18]:
orders_data.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [15]:
order_items_data.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93


In [20]:
order_items_data.nunique()

order_id               98666
order_item_id             21
product_id             32951
seller_id               3095
shipping_limit_date    93318
price                   5968
freight_value           6999
dtype: int64

In [21]:
order_items_data.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

### 1. Сколько пользователей, которые совершили покупку только один раз?

In [23]:
# группируем пользователей по уникальному id
# подсчитываем количество позаказных id пользователя
# выбираем пользователей, у которых количество заказов = 1
# выводим число строк
customers_data.groupby(['customer_unique_id'], as_index=False) \
.agg({'customer_id': 'count'})\
.rename(columns={'customer_id': 'order_count'})\
.query('order_count == 1')\
.shape[0]

93099

Таким образом, 93099 пользователей совершили 1 заказ. 
Посчитаем, какое количество пользователей совершили покупку 1 раз (т.е. есть дата оплаты, и заказ не отменен).


In [78]:
# мерджим датасет с пользователями, которые совершили 1 заказ с датасетом с уникальными идентификаторами пользователей
data_with_customer_id = pd.merge(customers_data.groupby(['customer_unique_id'], as_index=False)\
                .agg({'customer_id': 'count'})\
                .rename(columns={'customer_id': 'order_count'})\
                .query('order_count == 1'), customers_data[['customer_id','customer_unique_id']], on=['customer_unique_id'])

In [75]:
# мерджим датасеты с пользователями и заказами 
# удаляем строки, где не подтверждена оплата
# выбираем только те данные, где статус заказа не canceled
# выводим количество строк
pd.merge(data_with_customer_id, orders_data, on=['customer_id'])\
.dropna(subset=['order_approved_at'])\
.query('order_status != "canceled"')\
.shape[0]

92549

В результате, 93099 пользователей создали 1 заказ и 92549 пользователей оплатили 1 заказ. 

 ----

### 2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

Недоставленные заказы - заказы, статус которых не 'delivered'. 

Негативными событиями являются следующие статусы заказов:

   - unavailable (недоступен) 
   - canceled (отменён).

In [27]:
# создаем колонку с годом и месяцем создания заказа
orders_data['order_purchase_month'] = orders_data['order_purchase_timestamp'].apply(lambda x: x.strftime('%Y-%m'))

In [84]:
# выбираем данные, где статус заказа не delivered 
# группируем по статусу и по месяцу создания заказа
# считаем количество заказов 
# группируем по статусу заказа
# считаем среднее количество недоставленных заказов по причинам и округляем
orders_data[orders_data['order_status'].isin(['unavailable', 'canceled'])]\
.groupby(['order_status', 'order_purchase_month'], as_index=False)\
.agg({'order_id': 'count'})\
.groupby('order_status', as_index=False)\
.agg({'order_id': 'mean'})\
.rename(columns={'order_id': 'mean_order_count_in_month'})\
.round(2)\
.sort_values('mean_order_count_in_month', ascending=False)

,order_status,mean_order_count_in_month
1,unavailable,29.00
0,canceled,26.04


В среднем 29 заказов в месяц не доставляются по причине недоступности заказа (unavailable), 26.04 - по причине отмены (canceled). 

---

 ### 3. По каждому товару определить, в какой день недели товар чаще всего покупается. 

In [88]:
# создаем колонку с днями недели создания заказов
orders_data['day_of_week'] = orders_data['order_purchase_timestamp'].dt.day_name()

In [91]:
# мерджим таблицы с заказами и продуктами по номеру заказа
# группируем по продуктам и по дням недели 
# считаем количество заказов
data3 = pd.merge(orders_data, order_items_data, on=['order_id'])\
.groupby(['product_id', 'day_of_week'])\
.agg({'order_id': 'count'})

Необходимо найти для каждого продукта день недели с наибольшим количеством заказов, для этого:

In [32]:
# группируем по продукту
# находим максимальное значение заказов
max_order_count = data3.groupby('product_id')['order_id'].max().reset_index()

In [33]:
# мерджим датасеты с количеством заказов по продуктам и дням недели с максимальным количеством заказов 
products_by_day = pd.merge(data3, max_order_count, on=['product_id', 'order_id'])

In [101]:
products_by_day.head(2)

,product_id,day_of_week,order_id
0,00066f42aeeb9f3007548bb9d3f33c38,Sunday,1
1,00088930e925c41fd95ebfe695fd2655,Tuesday,1


In [102]:
# т.к. один товар может покупаться одинаково часто в несколько разных дней, сгрупируем данные по product_id
# создадим список из дней, в которые товар покупался наиболее часто
products_by_day.groupby('product_id').day_of_week.apply(list).to_frame().reset_index().head()

,product_id,day_of_week
0,00066f42aeeb9f3007548bb9d3f33c38,[Sunday]
1,00088930e925c41fd95ebfe695fd2655,[Tuesday]
2,0009406fd7479715e4bef61dd91f2462,[Thursday]
3,000b8f95fcb9e0096488278317764d19,"[Friday, Wednesday]"
4,000d9be29b5207b54e86aa1b1ac54872,[Tuesday]


----

 ### 4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)?  

In [35]:
# для решения задачи используем колонку order_purchase_month, созданную в прошлом задании 
# приводим колонку к типу datetime
orders_data['order_purchase_month'] = pd.to_datetime(orders_data['order_purchase_month'])

In [36]:
# мерджим таблицы с заказми и клиентами
# группируем по id клиента и по месяцам
# считаем количество заказов 
data4 = pd.merge(customers_data, orders_data, on=['customer_id'])\
.groupby(['customer_unique_id', 'order_purchase_month'],as_index=False)\
.agg({'order_id': 'count'})

In [37]:
# создадим колонку с количеством недель в месяце
data4['count_week_in_month'] = data4.order_purchase_month.dt.daysinmonth / 7

In [38]:
# посчитаем среднее количество заказов в неделю по месяцам
data4['mean_order_count_by_week'] = (data4.order_id / data4['count_week_in_month']).round(3)

In [39]:
data4.head()

,customer_unique_id,order_purchase_month,order_id,count_week_in_month,mean_order_count_by_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-01,1,4.428571,0.226
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-01,1,4.428571,0.226
2,0000f46a3911fa3c0805444483337064,2017-03-01,1,4.428571,0.226
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-01,1,4.428571,0.226
4,0004aac84e0df4da2b147fca70cf8255,2017-11-01,1,4.285714,0.233


----

 ### 5. Напиши функцию на python, позволяющую строить когортный анализ. В период с января по декабрь выяви когорту с самым высоким retention на 3-й месяц. 

In [5]:
def cohort_analysis(df, date, customer_id, n):
    """ Функция для построения когортного анализа (размер когорты - месяц, исследуемый показатель - retention)
    параметры:
          df - DateFrame
          date - дата первой покупки (pandas.Series)
          customer_id - id клиента (pandas.Series)
          n - количество месяцев в отчетном периоде (int)   
    """
    # создаем колонку с месяцами заказов
    df['order_month'] = df[date].dt.to_period('M')
    
    # создаем колонку с когортами: группируем пользователей по id, находим дату первой покупки
    df['cohort'] = df.groupby(customer_id)[date]\
                 .transform('min')\
                 .dt.to_period('M') 
    
    # записываем в новый df данные, сгруппированные по когортам и по месяцам заказов и с количеством уникальных пользоватей
    cohorts = df.groupby(['cohort', 'order_month'])\
    .agg({customer_id: pd.Series.nunique})\
    .reset_index(drop=False)
    
    # создаем колонку с количеством месяцев, прошедших с даты первой покупки
    cohorts['period_number'] = (cohorts.order_month - cohorts.cohort).apply(attrgetter('n'))
    
    # преобразуем таблицу с помощью pivot
    cohorts = cohorts.pivot_table(index='cohort',
                                  columns='period_number',
                                  values= customer_id)
    
    # создаем df retention - делим кол-во пользователей в когортах для каждого месяца на кол-во пользователей в нулевой месяц 
    retention = cohorts.divide(cohorts.iloc[:, 0], axis=0)
    
    # возвращаем df retention на заданное количество месяцев
    return retention.loc[:, 0:n] 
    

In [6]:
def cohort_analysis_visualization(df):
    """ Функция для визуализации когортного анализа (исследуемый показатель - retention)
    параметры:
          df - DateFrame            
    """
    final_retention = (df
            .style
            .set_caption('User retention by cohort')  # добавляем подпись
            .background_gradient(cmap='YlOrRd')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту
    return final_retention

In [7]:
# мерджим таблицы с клиентами и заказами
data5 = pd.merge(customers_data, orders_data, on=['customer_id'])

In [8]:
retention = cohort_analysis(data5, 'order_purchase_timestamp', 'customer_unique_id', 3)

In [11]:
cohort_analysis_visualization(retention)

period_number,0,1,2,3
cohort,,,,
2016-09,100.00%,,,
2016-10,100.00%,,,
2016-12,100.00%,100.00%,,
2017-01,100.00%,0.39%,0.26%,0.13%
2017-02,100.00%,0.23%,0.29%,0.11%
2017-03,100.00%,0.49%,0.38%,0.38%
2017-04,100.00%,0.60%,0.21%,0.17%
2017-05,100.00%,0.50%,0.50%,0.39%
2017-06,100.00%,0.48%,0.35%,0.41%


В период с января по декабрь 2017 года когорта с самым высоким retention на 3-й месяц:

In [112]:
print(retention[2].idxmax())

2017-09


---

### 6. Построй RFM-кластеры для пользователей. Выведи для каждого кластера средние значения метрик R, F, M (и опиши подробно, как были построены метрики R, F, M).

Рассчитаем метрики R, F, M:
   R - recency — давность (как давно ваши клиенты что-то покупали);
   F - frequency — частота (как часто клиенты покупают);  
   M - monetary — деньги (общая сумма покупок).
Для этого объединим датасеты с заказами, клиентами и продуктами:

In [46]:
# мерджим таблицы с заказами и клиентами и выбираем нужные столбцы
data6 = pd.merge(customers_data, orders_data, on='customer_id')
data6 = data6[['customer_unique_id', 'order_id', 'order_purchase_timestamp']]

In [47]:
# таблицу с продуктами группируем по Id заказа и суммируем цену(price)
data_product = order_items_data.groupby(['order_id'], as_index=False).agg({'price': sum})


In [48]:
# мерджим таблицы с клиентами и заказами и продуктами
data = pd.merge(data6, data_product, on='order_id' )

In [51]:
data.head(2)

,customer_unique_id,order_id,order_purchase_timestamp,price
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,2017-05-16 15:05:35,124.99
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,2018-01-12 20:48:24,289.00


Для расчета метрики R воспользуемся формулой: текущая дата - последняя дата заказа пользователя. 

Для расчета метрики M необходимо сгруппировать данные по клиентами и суммировать столбец price, для того чтобы расчитать общее количество денег, потраченных клиентом.

Для расчета F также нужно сгруппировать данные по клиентам и посчитать количество заказов, совершенных пользователем. 

Т.к. данные представлены за период до 18 года, вместо текущей даты будет использована самая поздняя дата заказа + 1 день: 

In [62]:
max_date = max(data.order_purchase_timestamp) +  dt.timedelta(days=1)

In [63]:
# записываем в df расчитанные метрики 
df_rfm = data.groupby('customer_unique_id', as_index=False)\
.agg({'order_purchase_timestamp': lambda x: (max_date - x.max()).days, 'order_id': 'count', 'price': 'sum'})\
.rename(columns= {'order_purchase_timestamp': 'recency', 'order_id': 'frequency', 'price': 'monetary'})\
.round(2)

In [64]:
df_rfm.head(2)

,customer_unique_id,recency,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,129.9
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,18.9


Далее строим RFM-кластеры. Для этого разбиваем каждую метрику на 3 группы.

Метрики recency и monetary разбиваем на основе квантилей, а для frequency задаем интервалы в ручную, т.к. более 75% пользователей совершили только 1 заказ.

In [66]:
df_rfm['R'] = pd.qcut(df_rfm['recency'], 3, ['3', '2', '1'])
df_rfm['F'] = pd.cut(df_rfm['frequency'], bins=[0, 1, 7, 20], labels=['1', '2', '3'])
df_rfm['M'] = pd.qcut(df_rfm['monetary'], 3, ['1', '2', '3'])

R: 

     1 - давние (> 296 дней),   
     2 - относительно недавние ('спящие') (154 - 296 дней),  
     3 - недавние (< 154 дней).
    
F: 

     1 - разовая (< 1),  
     2 - редкие (1 - 7),  
     3 - частые (> 7).    
    
M: 

     1 - низкий чек (< 58.9),  
     2 - средний чек (58.9 - 128.9),  
     3 - высокий чек (> 128.9). 
    

In [67]:
# Создаем колонку с сегментами
df_rfm['RFM'] = df_rfm['R'].astype('str') + df_rfm['F'].astype('str') + df_rfm['M'].astype('str')

Cредние значения метрик R, F, M для каждого сегмента:

In [68]:
mean_rfm = df_rfm.groupby('RFM', as_index=False)\
.agg({'recency': 'mean', 'frequency': 'mean', 'monetary': 'mean'})\
.round(2)

In [59]:
def cluster_description(x):
    """ Функция для описания сегментов (кластеров)
    параметры:
          x - номер сегмента           
    """
    
    k = ''
    if x[0] == '1':
        k += 'давние, '
    elif x[0] == '2':
         k += 'спящие, '
    else:
        k += 'недавние, '
        
        
    if x[1] == '1':
        k += 'разовая покупка, '
    elif x[1] == '2':
         k += 'редкие покупки, '
    else:
        k += 'частые покупки, '
        
        
    if x[2] == '1':
        k += 'низкий чек'
    elif x[2] == '2':
         k += 'средний чек'
    else:
        k += 'высокий чек'   
        
    return k

In [69]:
mean_rfm['cluster_description'] = mean_rfm['RFM'].apply(lambda x: cluster_description(x))

In [70]:
mean_rfm

,RFM,recency,frequency,monetary,cluster_description
0,111,426.48,1.00,34.64,"давние, разовая покупка, низкий чек"
1,112,424.17,1.00,87.61,"давние, разовая покупка, средний чек"
2,113,425.19,1.00,312.56,"давние, разовая покупка, высокий чек"
3,121,446.92,2.13,42.84,"давние, редкие покупки, низкий чек"
4,122,409.32,2.03,96.42,"давние, редкие покупки, средний чек"
5,123,407.00,2.11,358.66,"давние, редкие покупки, высокий чек"
6,211,225.66,1.00,35.11,"спящие, разовая покупка, низкий чек"
7,212,227.15,1.00,89.87,"спящие, разовая покупка, средний чек"
8,213,226.07,1.00,282.61,"спящие, разовая покупка, высокий чек"
9,221,223.72,2.02,43.82,"спящие, редкие покупки, низкий чек"
